In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Pré-processamento #

### Base e encoder manual ###

In [2]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df_encod_manual = pd.DataFrame.copy(df)

In [5]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [6]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [7]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [8]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [9]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [10]:
previsores_label = df.iloc[:, 0:11].values

In [11]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [12]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [13]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [14]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [15]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [16]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [17]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)